In [1]:
import io
import json
import datetime
import requests
import pandas as pd


data_url = 'https://api.statbank.dk/v1/data'
info_url = 'https://api.statbank.dk/v1/tableinfo/DNVALD?lang=en&format=JSON'

CUTOFF_PAST_DATE = datetime.date(year=2021, month=1, day=1)
CURRENCY_LIST = ['EUR', 'GBP', 'USD', 'SEK']

print('Will extract convertion rates starting from:', CUTOFF_PAST_DATE)

Will extract convertion rates starting from: 2021-01-01


In [2]:
info_resp = requests.get(url=info_url)
print('Status code:', info_resp.status_code)
if info_resp.status_code == 200:
    info_dict = json.loads(info_resp.text)
else:
    print(info_resp.text)

Status code: 200


In [3]:
# Checking that the info dict has the variable order we expect

assert info_dict['variables'][0]['id'].upper() == 'VALUTA', \
    'Expecting the first variable in info_dict to have id VALUTA'
assert info_dict['variables'][1]['id'].upper() == 'KURTYP', \
    'Expecting the second variable in info_dict to have id KURTYP'
assert info_dict['variables'][2]['id'].upper() == 'TID', \
    'Expecting the second variable in info_dict to have id TID'

In [4]:
valuta_df = pd.DataFrame.from_dict(info_dict['variables'][0]['values'])

# Check that our CURRENCY_LIST is contained in the available valuta
assert set(CURRENCY_LIST).issubset(set(valuta_df.id)), \
    f'Could not find currency: {set(CURRENCY_LIST).difference(set(valuta_df.id))}' 

# cursus_type_df = pd.DataFrame.from_dict(info_dict['variables'][1]['values'])
currency_days_df = pd.DataFrame.from_dict(info_dict['variables'][2]['values'])

# convert date text value like `2022M06D02` to a datetime and then extract the date component (.dt.date)
currency_days_df['date'] = pd.to_datetime(currency_days_df.id, format='%YM%mD%d').dt.date

dates_from_cutoff = currency_days_df[currency_days_df['date'] > CUTOFF_PAST_DATE].id.tolist()
assert len(dates_from_cutoff) > 0, f'Could not find any dates beyond the CUTOFF_PAST_DATE of {CUTOFF_PAST_DATE}'

In [5]:
data_params = {
    "lang": "en",
    "table": "DNVALD",
    "format": "CSV",
    "valuePresentation": "Code",
    "variables": [
      {
         "code": "VALUTA",
         "values": CURRENCY_LIST
      },
      {
         "code": "KURTYP",
         "values": [
            "KBH"
         ]
      },
      {
         "code": "TID",
         "values": dates_from_cutoff
      }
    ]
}

In [6]:
data_resp = requests.post(url=data_url, json=data_params)
print('Status code:', data_resp.status_code)
if data_resp.status_code != 200:
    print(data_resp.text)
else:
    csv_resp_io = io.StringIO(data_resp.text)    
    result_df = pd.read_csv(csv_resp_io, sep=';')
    result_df['date'] = pd.to_datetime(result_df['TID'], format='%YM%mD%d').dt.date

Status code: 200


In [7]:
currency_pivot_df = result_df.pivot_table(index=['date'], columns=['VALUTA'], values=['INDHOLD'])
#currency_pivot_df.plot()

In [8]:
currency_pivot_df

INDHOLD                       
VALUTA         EUR     GBP    SEK     USD
date                                     
2021-01-04  743.79  824.97  73.72  604.90
2021-01-05  743.87  823.48  73.97  606.20
2021-01-06  743.93  820.80  73.91  602.96
2021-01-07  743.92  824.84  73.97  606.00
2021-01-08  743.69  825.15  73.99  607.09
...            ...     ...    ...     ...
2022-06-16  743.86  869.50  69.56  715.25
2022-06-17  743.84  869.99  69.68  709.36
2022-06-20  743.87  867.51  69.93  707.30
2022-06-21  743.93  864.93  69.88  705.15
2022-06-22  743.87  866.12  69.72  707.03

[367 rows x 4 columns]

In [9]:
exportable_df = currency_pivot_df.droplevel(0, axis='columns').reset_index()
exportable_df.columns.name = None
exportable_df

,date,EUR,GBP,SEK,USD
0,2021-01-04,743.79,824.97,73.72,604.90
1,2021-01-05,743.87,823.48,73.97,606.20
2,2021-01-06,743.93,820.80,73.91,602.96
3,2021-01-07,743.92,824.84,73.97,606.00
4,2021-01-08,743.69,825.15,73.99,607.09
...,...,...,...,...,...
362,2022-06-16,743.86,869.50,69.56,715.25
363,2022-06-17,743.84,869.99,69.68,709.36
364,2022-06-20,743.87,867.51,69.93,707.30
365,2022-06-21,743.93,864.93,69.88,705.15


In [10]:
exportable_df.to_csv('exportable_df.csv')